# Download and Save the Model

To save this model for use in various locations, including other notebooks or a model server, upload it to S3-compatible storage.

## Install Required Packages

In [ ]:
!pip install boto3 botocore

## Download Model from Huggingface

In [ ]:
import os

git_repo = "https://huggingface.co/ibm-granite/granite-7b-instruct"
!git clone $git_repo

model_name = os.path.basename(git_repo)
print(f"Model Name: {model_name}")


## Configure Environment Variables

In [ ]:
import boto3
import botocore

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.getenv('AWS_S3_ENDPOINT')
region_name = os.getenv('AWS_DEFAULT_REGION')
bucket_name = os.getenv('AWS_S3_BUCKET')

session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name
)

bucket = s3_resource.Bucket(bucket_name)


## Upload Helper Functions

In [ ]:
def upload_directory_to_s3(local_directory, s3_prefix, remove_safetensors=True):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            if ".git" in relative_path:
                print(f"Skipping {relative_path}")
                continue
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"Uploading {file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    for obj in bucket.objects.filter(Prefix=prefix).all():
        print(obj.key)


## Upload and Verify

In [ ]:
# Upload model to S3
upload_directory_to_s3(model_name, f"models/{model_name}")

# Verify uploaded files
list_objects("models")


## Required Hardware by Module

| Module              | Required Hardware       |
|---------------------|---------------------------|
| Model Download      | Internet Connection      |
| Model Upload        | S3-Compatible Storage    |
| AWS Integration     | AWS Account & S3 Bucket |
| Large Models (>7B)  | High-Performance Storage |


### Next Steps

Now that the model is uploaded to S3 storage, you can reference the model using the same data connection and deploy it as an API.